In [2]:
#import datetime
#from dateutil.parser import parse

#from datetime import datetime, timedelta

#from google.api_core.exceptions import Conflict


import os
import requests
import pandas as pd
import reverse_geocoder as rg
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from datetime import date

# Creamos los permisos con la llave para ingresar a Google 
# activamos el servicio
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../notebooks/Clave_Google/henry-sismos-a343182ba163.json"

project_id = 'your-project-id'
client = bigquery.Client(project=project_id)



# Query para contar los valoree por cada actualizacion
SELECT Act, COUNT(Fecha) AS CountFecha
FROM `henry-sismos.Sismos.EEUU_sismo`
GROUP BY Act

# Creamos la ETL por Pais para google cloud

Ya teniendo la conexión y un código funcional, es el momento de aplicar la ETL ya desarrollada

## Colombia

In [6]:
# Creamos una tabla con los datos de la api en google cloud

# Obtener los datos de la API de USGS
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
parameters = {
    'format': 'geojson',
    'starttime': '1990-01-01',
    'endtime': '2023-03-28',
    'minmagnitude': '2.5',
    'minlatitude': '-5.266',
    'maxlatitude': '15.708',
    'minlongitude': '276.328',
    'maxlongitude': '293.906',
    'limit': 20000
}
response = requests.get(url, params=parameters)
data = response.json()

# Crear una lista vacía para almacenar los registros
records = []

# Recorrer los eventos y extraer los parámetros

    
for event in data['features']:
    properties = event['properties']
    coordinates = event['geometry']['coordinates']
    properties['latitude'] = coordinates[1]
    properties['longitude'] = coordinates[0]
    properties['hipocentro'] = coordinates[2]
    records.append(properties)

# Crear el DataFrame
df_Colombia = pd.DataFrame(records)

# Damos formato de fecha a la columna "time"
df_Colombia["time"] = pd.to_datetime(df_Colombia["time"], unit= "ms")


# Quitemos los nulos
df_Colombia = df_Colombia[["time", "mag", "cdi", "gap", "latitude", "longitude", "place", "ids", "hipocentro" ]]
df_Colombia = df_Colombia.dropna()



# Creamos las columnas con "Pais" y con "Region"

# Crear una lista de tuplas de coordenadas (latitud, longitud)
coordinates = list(zip(df_Colombia['latitude'], df_Colombia['longitude']))

# Realizar la búsqueda inversa de las coordenadas
results = rg.search(coordinates)

# Obtener la lista de prefecturas a partir de los resultados
prefectures = [result['admin1'] if result['admin1'] != '' else 'Desconocido' for result in results]
countries = [result['cc'] if result['cc'] != '' else 'Desconocido' for result in results]
# Agregar la columna "region" al DataFrame
df_Colombia['Region'] = prefectures
df_Colombia['Pais'] = countries

# Listado de valores a eliminar en la columna 'Col1'
paises_permitidos = ['CO']

# Eliminar las filas donde los valores de 'Pais' no coinciden con el listado
df_Colombia = df_Colombia[df_Colombia['Pais'].isin(paises_permitidos)]

# Renombramos las columnas
nuevos_nombres = {'time': "Fecha", 'mag': "Magnitud", 'cdi': "Intensidad",
                   'gap': "GAP", 'latitude': "Latitud", 'longitude': "Longitud", 
                   'place': "Lugar", 'ids': "Id", "hipocentro": 'Hipocentro', 'Region': 'Region' , 'Pais': 'Pais'}
df_Colombia.columns = nuevos_nombres.values()

# Creamos la columna de la actualización de la data 
df_Colombia["Act"] = pd.to_datetime(date.today()) 


# Creamos un archivo json con la data ya normalizada 
data = df_Colombia


# Configurar el proyecto, dataset y tabla
project_id = 'henry-sismos'
dataset_id = 'Sismos'
table_id = 'Colombia_sismo'

# Crear la tabla en BigQuery
client = bigquery.Client(project=project_id)
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

try:
    # Intentar obtener la tabla
    table = client.get_table(table_ref)
    print(f"La tabla {table_id} ya existe.")
except NotFound:
    # La tabla no existe, crearla
    schema = [
        bigquery.SchemaField('Fecha', 'TIMESTAMP'),
        bigquery.SchemaField('Magnitud', 'FLOAT'),
        bigquery.SchemaField('Intensidad', 'FLOAT'),
        bigquery.SchemaField('GAP', 'FLOAT'),
        bigquery.SchemaField('Latitud', 'FLOAT'),
        bigquery.SchemaField('Longitud', 'FLOAT'),
        bigquery.SchemaField('Hipocentro', 'FLOAT'),
        bigquery.SchemaField('Lugar', 'STRING'),
        bigquery.SchemaField('Id', 'STRING'),
        bigquery.SchemaField('Region', 'STRING'),
        bigquery.SchemaField('Pais', 'STRING'),
        bigquery.SchemaField('Act', 'TIMESTAMP')
    ]
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    # Cargar los datos en la tabla
    job_config = bigquery.LoadJobConfig(schema=schema)
    job = client.load_table_from_dataframe(df_Colombia, table_ref, job_config=job_config)
    print(f"La tabla {table_id} se ha creado correctamente.")

La tabla Colombia_sismo ya existe.


In [7]:
df_Colombia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395 entries, 2 to 4002
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Fecha       395 non-null    datetime64[ns]
 1   Magnitud    395 non-null    float64       
 2   Intensidad  395 non-null    float64       
 3   GAP         395 non-null    float64       
 4   Latitud     395 non-null    float64       
 5   Longitud    395 non-null    float64       
 6   Lugar       395 non-null    object        
 7   Id          395 non-null    object        
 8   Hipocentro  395 non-null    float64       
 9   Region      395 non-null    object        
 10  Pais        395 non-null    object        
 11  Act         395 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(6), object(4)
memory usage: 40.1+ KB


In [8]:
# Este es el código que se coloca en Google BigQuery
import base64
from google.cloud import bigquery
import requests
from datetime import datetime, timedelta
import pandas as pd
import reverse_geocoder as rg
def actualizar_colombia_sismo(request, context):
    project_id = 'henry-sismos'
    dataset_id = 'Sismos'
    table_id = 'Colombia_sismo'
    
    table_ref = f"{project_id}.{dataset_id}.{table_id}"

    # Crear el cliente de BigQuery
    client = bigquery.Client(project=project_id)

    # Obtener la fecha más reciente en la tabla "Colombia"
    latest_date_query = f"SELECT MAX(Fecha) FROM `{project_id}.{dataset_id}.{table_id}`"
    query_job = client.query(latest_date_query)
    latest_date_result = query_job.result()

    # Obtener el valor de la fecha más reciente
    for row in latest_date_result:
        latest_date = row[0]

    # Verificar si se obtuvo una fecha válida
    if latest_date is not None:
        latest_date = latest_date.date()
    else:
        # Si no hay fecha más reciente, establecer una fecha inicial
        latest_date = datetime(1990, 1, 1).date()

    # Obtener la fecha actual
    current_date = datetime.now().date()

    # Verificar si se necesita actualizar la tabla
    if current_date > latest_date:
        # Calcular la fecha de inicio y fin para la API
        start_date = latest_date + timedelta(days=1)
        end_date = current_date

        # Obtener los datos de la API de USGS
        url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
        parameters = {
            'format': 'geojson',
            'starttime': start_date.strftime('%Y-%m-%d'),
            'endtime': end_date.strftime('%Y-%m-%d'),
            'minmagnitude': '2.5',
            'minlatitude': '-5.266',
            'maxlatitude': '15.708',
            'minlongitude': '276.328',
            'maxlongitude': '293.906',
            'limit': 20000
        }
        response = requests.get(url, params=parameters)
        data = response.json()

        records = []

        # Recorrer los eventos y extraer los parámetros
        for event in data['features']:
            properties = event['properties']
            latitude = event['geometry']['coordinates'][1]
            longitude = event['geometry']['coordinates'][0]
            longitude2 = event['geometry']['coordinates'][2]
            properties['latitude'] = latitude
            properties['longitude'] = longitude
            properties['hipocentro'] = longitude2
            records.append(properties)

        # Crear el DataFrame
        df_Colombia = pd.DataFrame(records)
        
        # Dar formato de fecha a la columna "time"
        df_Colombia["time"] = pd.to_datetime(df_Colombia["time"], unit="ms")
        # Eliminar filas con valores nulos
        df_Colombia = df_Colombia[["time", "mag", "cdi", "gap", "latitude", "longitude", "place", "ids", "hipocentro"]]
        
        df_Colombia = df_Colombia.dropna()
        
        # Verificar si el DataFrame está vacío
        if not df_Colombia.empty:
            # Verificar la longitud de las listas de coordenadas
            if len(df_Colombia['latitude']) == len(df_Colombia['longitude']):
                

                # Crear una lista de tuplas de coordenadas (latitud, longitud)
                coordinates = list(zip(df_Colombia['latitude'], df_Colombia['longitude']))
                
                # Realizar la búsqueda inversa de las coordenadas
                results = rg.search(coordinates)

                # Obtener la lista de regiones y países a partir de los resultados
                regions = [result['admin1'] if result['admin1'] != '' else 'Desconocido' for result in results]
                countries = [result['cc'] if result['cc'] != '' else 'Desconocido' for result in results]
                
                # Agregar las columnas "Region" y "Pais" al DataFrame
                df_Colombia['Region'] = regions
                df_Colombia['Pais'] = countries
                
                # Listado de valores permitidos en la columna 'Pais'
                paises_permitidos = ['CO']

                # Filtrar las filas donde los valores de 'Pais' no coincidan con el listado
                df_Colombia = df_Colombia[df_Colombia['Pais'].isin(paises_permitidos)]

                # Renombrar las columnas
                nuevos_nombres = {'time': "Fecha", 'mag': "Magnitud", 'cdi': "Intensidad", 'gap': "GAP",
                                    'latitude': "Latitud", 'longitude': "Longitud", 'place': "Lugar", 'ids': "Id" , "hipocentro": 'Hipocentro',
                                    'Region': 'Region', 'Pais': 'Pais'}
                df_Colombia.rename(columns=nuevos_nombres, inplace=True)

                # Creamos la columna de la actualización de la data 
                df_Colombia["Act"] = pd.to_datetime(date.today()) 


                # Insertar los datos en la tabla si hay nuevos datos
                if not df_Colombia.empty:
                    # Definir el esquema para la tabla de BigQuery
                    schema = [
                        bigquery.SchemaField("Fecha", "TIMESTAMP"),
                        bigquery.SchemaField("Magnitud", "FLOAT"),
                        bigquery.SchemaField("Intensidad", "FLOAT"),
                        bigquery.SchemaField("GAP", "FLOAT"),
                        bigquery.SchemaField("Latitud", "FLOAT"),
                        bigquery.SchemaField("Longitud", "FLOAT"),
                        bigquery.SchemaField('Hipocentro', 'FLOAT'),
                        bigquery.SchemaField('Lugar', 'STRING'),
                        bigquery.SchemaField('Id', 'STRING'),
                        bigquery.SchemaField('Region', 'STRING'),
                        bigquery.SchemaField('Pais', 'STRING'),
                        bigquery.SchemaField('Act', 'TIMESTAMP')
                    ]

                    # Actualizar la tabla en BigQuery
                    job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_APPEND")
                    job = client.load_table_from_dataframe(df_Colombia, table_ref, job_config=job_config)
                    print("La tabla ha sido actualizada.")
                else:
                    print("No hay nuevos datos para insertar.")

                
                
            else:
                print("La longitud de las listas de coordenadas no coincide.")
        else:
            print("El DataFrame df_Colombia está vacío.")
    else:
        print("La tabla ya está actualizada.")

### requirements.txt
google-cloud-bigquery <br>
pandas <br>
reverse_geocoder <br>
requests <br>
pyarrow <br>

# Japan

In [9]:
# Creamos una tabla con los datos de la api en google cloud

# Obtener los datos de la API de USGS
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
parameters = {
    'format': 'geojson',
    'starttime': '1990-01-01',
    'endtime': '2023-03-28',
    'minmagnitude': '2.5',
    'minlatitude': '28.691',
    'maxlatitude': '47.458',
    'minlongitude': '125.859',
    'maxlongitude': '156.445',
    'limit': 20000
}
response = requests.get(url, params=parameters)
data = response.json()

# Crear una lista vacía para almacenar los registros
records = []

# Recorrer los eventos y extraer los parámetros


for event in data['features']:
    properties = event['properties']
    coordinates = event['geometry']['coordinates']
    properties['latitude'] = coordinates[1]
    properties['longitude'] = coordinates[0]
    properties['hipocentro'] = coordinates[2]
    records.append(properties)

# Crear el DataFrame
df_Japan = pd.DataFrame(records)

# Damos formato de fecha a la columna "time"
df_Japan["time"] = pd.to_datetime(df_Japan["time"], unit= "ms")


# Quitemos los nulos
df_Japan = df_Japan[["time", "mag", "cdi", "gap", "latitude", "longitude", "place", "ids", "hipocentro" ]]
df_Japan = df_Japan.dropna()



# Creamos las columnas con "Pais" y con "Region"

# Crear una lista de tuplas de coordenadas (latitud, longitud)
coordinates = list(zip(df_Japan['latitude'], df_Japan['longitude']))

# Realizar la búsqueda inversa de las coordenadas
results = rg.search(coordinates)

# Obtener la lista de prefecturas a partir de los resultados
prefectures = [result['admin1'] if result['admin1'] != '' else 'Desconocido' for result in results]
countries = [result['cc'] if result['cc'] != '' else 'Desconocido' for result in results]
# Agregar la columna "region" al DataFrame
df_Japan['Region'] = prefectures
df_Japan['Pais'] = countries

# Listado de valores a eliminar en la columna 'Col1'
paises_permitidos = ['JP']

# Eliminar las filas donde los valores de 'Pais' no coinciden con el listado
df_Japan = df_Japan[df_Japan['Pais'].isin(paises_permitidos)]

# Renombramos las columnas
nuevos_nombres = {'time': "Fecha", 'mag': "Magnitud", 'cdi': "Intensidad",
                   'gap': "GAP", 'latitude': "Latitud", 'longitude': "Longitud", 
                   'place': "Lugar", 'ids': "Id", "hipocentro": 'Hipocentro', 'Region': 'Region' , 'Pais': 'Pais'}
df_Japan.columns = nuevos_nombres.values()

# Creamos la columna de la actualización de la data 
df_Japan["Act"] = pd.to_datetime(date.today()) 


# Creamos un archivo json con la data ya normalizada 
data = df_Japan


# Configurar el proyecto, dataset y tabla
project_id = 'henry-sismos'
dataset_id = 'Sismos'
table_id = 'Japan_sismo'

# Crear la tabla en BigQuery
client = bigquery.Client(project=project_id)
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

try:
    # Intentar obtener la tabla
    table = client.get_table(table_ref)
    print(f"La tabla {table_id} ya existe.")
except NotFound:
    # La tabla no existe, crearla
    schema = [
        bigquery.SchemaField('Fecha', 'TIMESTAMP'),
        bigquery.SchemaField('Magnitud', 'FLOAT'),
        bigquery.SchemaField('Intensidad', 'FLOAT'),
        bigquery.SchemaField('GAP', 'FLOAT'),
        bigquery.SchemaField('Latitud', 'FLOAT'),
        bigquery.SchemaField('Longitud', 'FLOAT'),
        bigquery.SchemaField('Hipocentro', 'FLOAT'),
        bigquery.SchemaField('Lugar', 'STRING'),
        bigquery.SchemaField('Id', 'STRING'),
        bigquery.SchemaField('Region', 'STRING'),
        bigquery.SchemaField('Pais', 'STRING'),
        bigquery.SchemaField('Act', 'TIMESTAMP')
    ]
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    # Cargar los datos en la tabla
    job_config = bigquery.LoadJobConfig(schema=schema)
    job = client.load_table_from_dataframe(df_Japan, table_ref, job_config=job_config)
    print(f"La tabla {table_id} se ha creado correctamente.")

La tabla Japan_sismo ya existe.


In [10]:
# Esta fue la función que se implemento e google cloud functions carga incremental o actualizacion de informacion

# Este es el código que se coloca en Google BigQuery
import base64
from google.cloud import bigquery
import requests
from datetime import datetime, timedelta, date
import pandas as pd
import reverse_geocoder as rg
def actualizar_japan_sismo(request, context):
    project_id = 'henry-sismos'
    dataset_id = 'Sismos'
    table_id = 'Japan_sismo'
    
    table_ref = f"{project_id}.{dataset_id}.{table_id}"

    # Crear el cliente de BigQuery
    client = bigquery.Client(project=project_id)

    # Obtener la fecha más reciente en la tabla "Colombia"
    latest_date_query = f"SELECT MAX(Fecha) FROM `{project_id}.{dataset_id}.{table_id}`"
    query_job = client.query(latest_date_query)
    latest_date_result = query_job.result()

    # Obtener el valor de la fecha más reciente
    for row in latest_date_result:
        latest_date = row[0]

    # Verificar si se obtuvo una fecha válida
    if latest_date is not None:
        latest_date = latest_date.date()
    else:
        # Si no hay fecha más reciente, establecer una fecha inicial
        latest_date = datetime(1990, 1, 1).date()

    # Obtener la fecha actual
    current_date = datetime.now().date()

    # Verificar si se necesita actualizar la tabla
    if current_date > latest_date:
        # Calcular la fecha de inicio y fin para la API
        start_date = latest_date + timedelta(days=1)
        end_date = current_date

        # Obtener los datos de la API de USGS
        url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
        parameters = {
            'format': 'geojson',
            'starttime': start_date.strftime('%Y-%m-%d'),
            'endtime': end_date.strftime('%Y-%m-%d'),
            'minmagnitude': '2.5',
            'minlatitude': '28.691',
            'maxlatitude': '47.458',
            'minlongitude': '125.859',
            'maxlongitude': '156.445',
            'limit': 20000
        }
        response = requests.get(url, params=parameters)
        data = response.json()

        records = []

        # Recorrer los eventos y extraer los parámetros
        for event in data['features']:
            properties = event['properties']
            latitude = event['geometry']['coordinates'][1]
            longitude = event['geometry']['coordinates'][0]
            longitude2 = event['geometry']['coordinates'][2]
            properties['latitude'] = latitude
            properties['longitude'] = longitude
            properties['hipocentro'] = longitude2
            records.append(properties)

        # Crear el DataFrame
        df_Japan = pd.DataFrame(records)
        
        # Dar formato de fecha a la columna "time"
        df_Japan["time"] = pd.to_datetime(df_Japan["time"], unit="ms")
        # Eliminar filas con valores nulos
        df_Japan = df_Japan[["time", "mag", "cdi", "gap", "latitude", "longitude", "place", "ids", "hipocentro"]]
        
        df_Japan = df_Japan.dropna()
        
        # Verificar si el DataFrame está vacío
        if not df_Japan.empty:
            # Verificar la longitud de las listas de coordenadas
            if len(df_Japan['latitude']) == len(df_Japan['longitude']):
                

                # Crear una lista de tuplas de coordenadas (latitud, longitud)
                coordinates = list(zip(df_Japan['latitude'], df_Japan['longitude']))
                
                # Realizar la búsqueda inversa de las coordenadas
                results = rg.search(coordinates)

                # Obtener la lista de regiones y países a partir de los resultados
                regions = [result['admin1'] if result['admin1'] != '' else 'Desconocido' for result in results]
                countries = [result['cc'] if result['cc'] != '' else 'Desconocido' for result in results]
                
                # Agregar las columnas "Region" y "Pais" al DataFrame
                df_Japan['Region'] = regions
                df_Japan['Pais'] = countries
                
                # Listado de valores permitidos en la columna 'Pais'
                paises_permitidos = ['JP']

                # Filtrar las filas donde los valores de 'Pais' no coincidan con el listado
                df_Japan = df_Japan[df_Japan['Pais'].isin(paises_permitidos)]

                # Renombrar las columnas
                nuevos_nombres = {'time': "Fecha", 'mag': "Magnitud", 'cdi': "Intensidad", 'gap': "GAP",
                                    'latitude': "Latitud", 'longitude': "Longitud", 'place': "Lugar", 'ids': "Id" , "hipocentro": 'Hipocentro',
                                    'Region': 'Region', 'Pais': 'Pais'}
                df_Japan.rename(columns=nuevos_nombres, inplace=True)

                # Creamos la columna de la actualización de la data 
                df_Japan["Act"] = pd.to_datetime(date.today()) 


                # Insertar los datos en la tabla si hay nuevos datos
                if not df_Japan.empty:
                    # Definir el esquema para la tabla de BigQuery
                    schema = [
                        bigquery.SchemaField("Fecha", "TIMESTAMP"),
                        bigquery.SchemaField("Magnitud", "FLOAT"),
                        bigquery.SchemaField("Intensidad", "FLOAT"),
                        bigquery.SchemaField("GAP", "FLOAT"),
                        bigquery.SchemaField("Latitud", "FLOAT"),
                        bigquery.SchemaField("Longitud", "FLOAT"),
                        bigquery.SchemaField('Hipocentro', 'FLOAT'),
                        bigquery.SchemaField('Lugar', 'STRING'),
                        bigquery.SchemaField('Id', 'STRING'),
                        bigquery.SchemaField('Region', 'STRING'),
                        bigquery.SchemaField('Pais', 'STRING'),
                        bigquery.SchemaField('Act', 'TIMESTAMP')
                    ]

                    # Actualizar la tabla en BigQuery
                    job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_APPEND")
                    job = client.load_table_from_dataframe(df_Japan, table_ref, job_config=job_config)
                    print("La tabla ha sido actualizada.")
                else:
                    print("No hay nuevos datos para insertar.")

                
                
            else:
                print("La longitud de las listas de coordenadas no coincide.")
        else:
            print("El DataFrame df_Japan está vacío.")
    else:
        print("La tabla ya está actualizada.")

### requirements.txt
google-cloud-bigquery <br>
pandas <br>
reverse_geocoder <br>
requests <br>
pyarrow <br>

# EEUU

In [11]:
# Creamos una tabla con los datos de la api en google cloud

# Obtener los datos de la API de USGS


url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
parameters = {
    'format': 'geojson',
    'starttime': '1990-01-01',
    'endtime': '2023-03-28',
    'minmagnitude': '2.5',
    'minlatitude': '12.555',
    'maxlatitude': '75.141',
    'minlongitude': '182.813',
    'maxlongitude': '315',
    'limit': 20000
}
response = requests.get(url, params=parameters)
data = response.json()

# Crear una lista vacía para almacenar los registros
records = []

# Recorrer los eventos y extraer los parámetros

for event in data['features']:
    properties = event['properties']
    coordinates = event['geometry']['coordinates']
    properties['latitude'] = coordinates[1]
    properties['longitude'] = coordinates[0]
    properties['hipocentro'] = coordinates[2]
    records.append(properties)

# Crear el DataFrame
df_EEUU = pd.DataFrame(records)

# Damos formato de fecha a la columna "time"
df_EEUU["time"] = pd.to_datetime(df_EEUU["time"], unit= "ms")



# Quitemos los nulos
df_EEUU = df_EEUU[["time", "mag", "cdi", "gap", "latitude", "longitude", "place", "ids", "hipocentro" ]]
df_EEUU = df_EEUU.dropna()




# Creamos las columnas con "Pais" y con "Region"

# Crear una lista de tuplas de coordenadas (latitud, longitud)
coordinates = list(zip(df_EEUU['latitude'], df_EEUU['longitude']))

# Realizar la búsqueda inversa de las coordenadas
results = rg.search(coordinates)

# Obtener la lista de prefecturas a partir de los resultados
prefectures = [result['admin1'] if result['admin1'] != '' else 'Desconocido' for result in results]
countries = [result['cc'] if result['cc'] != '' else 'Desconocido' for result in results]
# Agregar la columna "region" al DataFrame
df_EEUU['Region'] = prefectures
df_EEUU['Pais'] = countries

# Listado de valores a eliminar en la columna 'Col1'
paises_permitidos = ['US']

# Eliminar las filas donde los valores de 'Pais' no coinciden con el listado
df_EEUU = df_EEUU[df_EEUU['Pais'].isin(paises_permitidos)]


# Renombramos las columnas
nuevos_nombres = {'time': "Fecha", 'mag': "Magnitud", 'cdi': "Intensidad",
                   'gap': "GAP", 'latitude': "Latitud", 'longitude': "Longitud", 
                   'place': "Lugar", 'ids': "Id", "hipocentro": 'Hipocentro', 'Region': 'Region' , 'Pais': 'Pais'}
df_EEUU.columns = nuevos_nombres.values()

# Creamos la columna de la actualización de la data 
df_EEUU["Act"] = pd.to_datetime(date.today()) 


# Creamos un archivo json con la data ya normalizada 
data = df_EEUU


# Configurar el proyecto, dataset y tabla
project_id = 'henry-sismos'
dataset_id = 'Sismos'
table_id = 'EEUU_sismo'

# Crear la tabla en BigQuery
client = bigquery.Client(project=project_id)
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

try:
    # Intentar obtener la tabla
    table = client.get_table(table_ref)
    print(f"La tabla {table_id} ya existe.")
except NotFound:
    # La tabla no existe, crearla
    schema = [
        bigquery.SchemaField('Fecha', 'TIMESTAMP'),
        bigquery.SchemaField('Magnitud', 'FLOAT'),
        bigquery.SchemaField('Intensidad', 'FLOAT'),
        bigquery.SchemaField('GAP', 'FLOAT'),
        bigquery.SchemaField('Latitud', 'FLOAT'),
        bigquery.SchemaField('Longitud', 'FLOAT'),
        bigquery.SchemaField('Hipocentro', 'FLOAT'),
        bigquery.SchemaField('Lugar', 'STRING'),
        bigquery.SchemaField('Id', 'STRING'),
        bigquery.SchemaField('Region', 'STRING'),
        bigquery.SchemaField('Pais', 'STRING'),
        bigquery.SchemaField('Act', 'TIMESTAMP')
    ]
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    # Cargar los datos en la tabla
    job_config = bigquery.LoadJobConfig(schema=schema)
    job = client.load_table_from_dataframe(df_EEUU, table_ref, job_config=job_config)
    print(f"La tabla {table_id} se ha creado correctamente.")








                        Fecha  Magnitud  Intensidad    GAP    Latitud   
2     2023-03-27 04:53:13.815      4.00         3.8  199.0  43.344600  \
4     2023-03-27 04:39:16.340      4.12         3.8  276.0  18.815500   
12    2023-03-21 23:18:19.630      4.54         4.1  204.0  40.541500   
14    2023-03-20 23:25:28.920      4.90         3.8   32.0  53.790300   
29    2023-03-17 03:33:15.894      4.20         2.5  145.0  52.626100   
...                       ...       ...         ...    ...        ...   
19860 2007-01-03 14:34:38.540      4.40         3.2   42.9  37.067000   
19891 2006-12-24 03:43:38.860      4.11         4.3   33.0  33.706833   
19908 2006-12-16 06:14:05.350      4.20         4.1   69.0  36.173000   
19950 2006-11-29 21:10:55.430      4.11         5.4   48.0  32.820500   
19979 2006-11-23 10:42:57.420      4.30         3.9   40.7  37.157000   

         Longitud                           Lugar   
2     -126.896100      203 km W of Bandon, Oregon  \
4     -155.160333

In [12]:
# Esta fue la función que se implemento e google cloud functions carga incremental o actualizacion de informacion

import base64
from google.cloud import bigquery
import requests
from datetime import datetime, timedelta, date
import pandas as pd
import reverse_geocoder as rg
def actualizar_EEUU_sismo(request, context):
    project_id = 'henry-sismos'
    dataset_id = 'Sismos'
    table_id = 'EEUU_sismo'
    
    table_ref = f"{project_id}.{dataset_id}.{table_id}"

    # Crear el cliente de BigQuery
    client = bigquery.Client(project=project_id)

    # Obtener la fecha más reciente en la tabla "Colombia"
    latest_date_query = f"SELECT MAX(Fecha) FROM `{project_id}.{dataset_id}.{table_id}`"
    query_job = client.query(latest_date_query)
    latest_date_result = query_job.result()

    # Obtener el valor de la fecha más reciente
    for row in latest_date_result:
        latest_date = row[0]

    # Verificar si se obtuvo una fecha válida
    if latest_date is not None:
        latest_date = latest_date.date()
    else:
        # Si no hay fecha más reciente, establecer una fecha inicial
        latest_date = datetime(1990, 1, 1).date()

    # Obtener la fecha actual
    current_date = datetime.now().date()

    # Verificar si se necesita actualizar la tabla
    if current_date > latest_date:
        # Calcular la fecha de inicio y fin para la API
        start_date = latest_date + timedelta(days=1)
        end_date = current_date

        # Obtener los datos de la API de USGS
        url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
        parameters = {
            'format': 'geojson',
            'starttime': start_date.strftime('%Y-%m-%d'),
            'endtime': end_date.strftime('%Y-%m-%d'),
            'minmagnitude': '2.5',
            'minlatitude': '12.555',
            'maxlatitude': '75.141',
            'minlongitude': '182.813',
            'maxlongitude': '315',
            'limit': 20000
        }
        response = requests.get(url, params=parameters)
        data = response.json()

        records = []

        # Recorrer los eventos y extraer los parámetros
        for event in data['features']:
            properties = event['properties']
            latitude = event['geometry']['coordinates'][1]
            longitude = event['geometry']['coordinates'][0]
            longitude2 = event['geometry']['coordinates'][2]
            properties['latitude'] = latitude
            properties['longitude'] = longitude
            properties['hipocentro'] = longitude2
            records.append(properties)

        # Crear el DataFrame
        df_EEUU = pd.DataFrame(records)
        
        # Dar formato de fecha a la columna "time"
        df_EEUU["time"] = pd.to_datetime(df_EEUU["time"], unit="ms")
        # Eliminar filas con valores nulos
        df_EEUU = df_EEUU[["time", "mag", "cdi", "gap", "latitude", "longitude", "place", "ids", "hipocentro"]]
        
        df_EEUU = df_EEUU.dropna()
        
        # Verificar si el DataFrame está vacío
        if not df_EEUU.empty:
            # Verificar la longitud de las listas de coordenadas
            if len(df_EEUU['latitude']) == len(df_EEUU['longitude']):
                

                # Crear una lista de tuplas de coordenadas (latitud, longitud)
                coordinates = list(zip(df_EEUU['latitude'], df_EEUU['longitude']))
                
                # Realizar la búsqueda inversa de las coordenadas
                results = rg.search(coordinates)

                # Obtener la lista de regiones y países a partir de los resultados
                regions = [result['admin1'] if result['admin1'] != '' else 'Desconocido' for result in results]
                countries = [result['cc'] if result['cc'] != '' else 'Desconocido' for result in results]
                
                # Agregar las columnas "Region" y "Pais" al DataFrame
                df_EEUU['Region'] = regions
                df_EEUU['Pais'] = countries
                
                # Listado de valores permitidos en la columna 'Pais'
                paises_permitidos = ['US']

                # Filtrar las filas donde los valores de 'Pais' no coincidan con el listado
                df_EEUU = df_EEUU[df_EEUU['Pais'].isin(paises_permitidos)]

                # Renombrar las columnas
                nuevos_nombres = {'time': "Fecha", 'mag': "Magnitud", 'cdi': "Intensidad", 'gap': "GAP",
                                    'latitude': "Latitud", 'longitude': "Longitud", 'place': "Lugar", 'ids': "Id" , "hipocentro": 'Hipocentro',
                                    'Region': 'Region', 'Pais': 'Pais'}
                df_EEUU.rename(columns=nuevos_nombres, inplace=True)

                # Creamos la columna de la actualización de la data 
                df_EEUU["Act"] = pd.to_datetime(date.today()) 


                # Insertar los datos en la tabla si hay nuevos datos
                if not df_EEUU.empty:
                    # Definir el esquema para la tabla de BigQuery
                    schema = [
                        bigquery.SchemaField("Fecha", "TIMESTAMP"),
                        bigquery.SchemaField("Magnitud", "FLOAT"),
                        bigquery.SchemaField("Intensidad", "FLOAT"),
                        bigquery.SchemaField("GAP", "FLOAT"),
                        bigquery.SchemaField("Latitud", "FLOAT"),
                        bigquery.SchemaField("Longitud", "FLOAT"),
                        bigquery.SchemaField('Hipocentro', 'FLOAT'),
                        bigquery.SchemaField('Lugar', 'STRING'),
                        bigquery.SchemaField('Id', 'STRING'),
                        bigquery.SchemaField('Region', 'STRING'),
                        bigquery.SchemaField('Pais', 'STRING'),
                        bigquery.SchemaField('Act', 'TIMESTAMP')
                    ]

                    # Actualizar la tabla en BigQuery
                    job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_APPEND")
                    job = client.load_table_from_dataframe(df_EEUU, table_ref, job_config=job_config)
                    print("La tabla ha sido actualizada.")
                else:
                    print("No hay nuevos datos para insertar.")

                
                
            else:
                print("La longitud de las listas de coordenadas no coincide.")
        else:
            print("El DataFrame df_EEUU está vacío.")
    else:
        print("La tabla ya está actualizada.")